In [ ]:
import pandas as pd
from pathlib import Path
import glob
import sqlite3

path = {
    'facebook':{
        'post':'..\\data\\facebook-news-master\\fb_news_posts_20K.csv',
        'comment':'..\\data\\facebook-news-master\\fb_news_comments_1000K_hashed.csv',
        'source':'..\\data\\facebook-news-master\\fb_news_pagenames.csv'
        },
    'twitter':glob.glob('..\\data\\breaking-news-tweet\\*.csv'),
    'clean':'..\\data\\clean',
    'database':'..\\database\\news.db'
}

Path('..\\data\\clean').mkdir(parents=True, exist_ok=True)


# Twitter

In [ ]:
frame = None
for index, filename in enumerate(path['twitter']):
    new_frame = pd.read_csv(filename, index_col=9, dtype={'tweetid':str, 'quoted_status_id':str})\
        .query("verified and language.str.lower() == 'en'")

    if frame is None:
        frame = new_frame.copy(deep=True)
    else:
        frame = pd.concat([frame, new_frame])

In [ ]:
# filtered_verified_en = frame.query("verified and language.str.lower() == 'en'")
frame.to_csv(f'{path["clean"]}\\twitter_verified_en.csv')

# filtered_en = frame.query("language.str.lower() == 'en'")
# filtered_en.to_csv(f'{path["clean"]}\\twitter_en.csv')

print(len(frame))

In [ ]:
filtered_col = frame.loc[:,['hashed_userid','location','tweetcreatedts','text']]
filtered_col.to_csv(f'{path["clean"]}\\tweet_clean.csv')

In [ ]:
with sqlite3.connect(path['database']) as conn:
    filtered_col.to_sql('twitter', conn, if_exists='replace')

# Facebook

In [ ]:
posts = pd.read_csv(path['facebook']['post'])
comments = pd.read_csv(path['facebook']['comment'], index_col=[0,1])
sources = pd.read_csv(path['facebook']['source'], index_col=1)

In [ ]:
posts_filtered = posts.loc[:,['created_time', 'link', 'message', 'page_id', 'post_id']]
posts_filtered['post_id'] = posts_filtered['post_id'].apply(lambda value: value.split('_')[-1])
posts_filtered.index = posts_filtered['post_id']
posts_filtered = posts_filtered.iloc[:,:-1]
posts_filtered.to_csv(f'{path["clean"]}\\facebook_posts.csv')

comments_filtered = comments.loc[:, ['message', 'post_name']]
comments_filtered['post_id'] = comments_filtered['post_name'].apply(lambda value: value.split('_')[-1])
comments_filtered.to_csv(f'{path["clean"]}\\facebook_comments.csv')

In [ ]:
with sqlite3.connect(path['database']) as conn:
    posts_filtered.to_sql('post', conn, if_exists='replace')
    comments_filtered.to_sql('comment', conn, if_exists='replace')

In [ ]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

trust_sources = [
    '228735667216',
    '15704546335',
    '86680728811',
    '155869377766434',
    '131459315949',
    '5550296508',
    '6250307292',
    '5281959998',
    '6013004059',
    '8860325749',
    '10513336322',
    '164305410295882',
    '268914272540',
    '18468761129',
    '10606591490',
    '7382473689',
    '273864989376427',
    '10643211755'
]
with sqlite3.connect(path['database']) as conn:
    conn.row_factory = dict_factory
    cur = conn.cursor()
    comments_clean = cur.execute(f"""
        select c.* from comment c inner join post p on c.post_id = p.post_id where p.page_id in(
        {','.join(trust_sources)}
        );
    """).fetchall()
    posts_clean = cur.execute(f"""
        select p.* from post p where p.page_id in(
        {','.join(trust_sources)}
        );
    """).fetchall()
comments_clean = pd.DataFrame(comments_clean).loc[:, ['from_id', 'post_id', 'message', 'created_time']]
posts_clean = pd.DataFrame(posts_clean)

comments_clean.index = comments_clean['created_time']
comments_clean = comments_clean.iloc[:, :-1]
comments_clean.to_csv(f'{path["clean"]}\\facebook_comments_clean.csv')

posts_clean.index = posts_clean['post_id']
posts_clean = posts_clean.iloc[:, 1:]
posts_clean.to_csv(f'{path["clean"]}\\facebook_posts_clean.csv')